## Subject-level analysis pipeline for FTT-fNIRs-data

<font color='blue'>Version 1.0 (2023-06-18),<br>Version 2.0 (2023-07-08),<br>Version 2.1 (2023-07-09),<br>Version 2.2 (2023-07-10),<br></font> Dr. Torsten Wüstenberg (CNSR)

**Aim:** Preprocessing, artifact reduction and subject-level modeling of data for a single fNIRS measurement.

**Gereral analysis steps:**

1. *Specify data for analysis*
2. *Load data for analysis*
3. *Convert raw data values into optical densities*
4. *Data quality assessment: </br>- compute and assess scalp coupling index</br>- semi automatic rejection of bad channels*
5. *Artefact reduction*
6. *Converting from optical density to haemoglobin*
7. *Modelling:</br>- model specification</br>- model fitting</br>- writing model parameter estimates to table*

## Preparations ---------------------------------------------------------------------

### Import libraries
The analysis pipeline is based on the following libraries. In case of an error in the execution of this cell, probably one or more of the libraries is not installed. In this case, start a terminal in **ANACONDS.NAVIGATOR** *Environments>Terminal* and install the library in question using the command </br><font color='red'>*pip install [library name]*</font>.

In [ ]:
import warnings  # switch of pandas and mne warnings

warnings.simplefilter(action="ignore")
import matplotlib.pyplot as plt  # general plotting functions

%matplotlib qt
import mne  # core library for neurophysiological data analyses
import numpy as np  # general mathematical functions
import pandas as pd  # general table functions
import os  # general file and directory handling functions
from itertools import compress  # general iteration tool

# fNIRS specific mne-objercts
import mne_nirs
from mne_nirs.experimental_design import make_first_level_design_matrix
from mne_nirs.statistics import run_glm
from nilearn.plotting import plot_design_matrix
from nilearn.glm import first_level

### Define ROIs
Optical channels, pooled for different regions of interest (ROIs)

In [ ]:
leftMotor = [
    "S1_D1 hbo",
    "S1_D2 hbo",
    "S1_D3 hbo",
    "S2_D1 hbo",
    "S2_D3 hbo",
    "S2_D4 hbo",
    "S3_D2 hbo",
    "S3_D3 hbo",
    "S4_D3 hbo",
    "S4_D4 hbo",
]
rightMotor = [
    "S5_D5 hbo",
    "S5_D6 hbo",
    "S5_D7 hbo",
    "S6_D5 hbo",
    "S6_D7 hbo",
    "S6_D8 hbo",
    "S7_D6 hbo",
    "S7_D7 hbo",
    "S8_D7 hbo",
    "S8_D8 hbo",
]
leftFrontal = [
    "S9_D9 hbo",
    "S9_D10 hbo",
    "S10_D9 hbo",
    "S10_D11 hbo",
    "S11_D9 hbo",
    "S11_D11 hbo",
    "S11_D12 hbo",
    "S12_D10 hbo",
    "S13_D11 hbo",
]
rightFrontal = [
    "S12_D13 hbo",
    "S13_D14 hbo",
    "S14_D12 hbo",
    "S14_D14 hbo",
    "S14_D15 hbo",
    "S15_D13 hbo",
    "S15_D15 hbo",
    "S16_D14 hbo",
    "S16_D15 hbo",
]
medialFrontal = ["S12_D12 hbo", "S13_D12 hbo"]

### Prepare results output structure
Data frame for model parameter estimates

In [ ]:
fNIRSresHbO = pd.read_excel("FTT_fNIRS_results.xlsx")

## Pipeline ---------------------------------------------------------------------------------
### 1. Specify data for analysis
Three inputs are required! All Inputs are case sensitive! Directories with subject data should be in the same directory as the notebook and having the following structure:</br>
SID</br>
 |-HIIT    > data folders</br>
 |-nonHIIT > data folders

In [ ]:
sid = input("SID:")
condition = input("Condition (HIIT, nonHIIT): ")
session = input("Session ([T]raining, [R]etention, [C]ontrol): ")

In [ ]:
dataDir = (
    os.getcwd()
    + os.sep
    + "Data"
    + os.sep
    + sid
    + os.sep
    + condition
    + os.sep
    + fNIRSresHbO[fNIRSresHbO.SID == sid][condition + "(" + session + ")"].to_string(
        index=False
    )
)
dataDir

### 2. Loading data for analysis

In [ ]:
raw_intensity = mne.io.read_raw_nirx(dataDir, verbose=True)
raw_intensity.load_data()

Set block duration and show raw data (only for visual sanity check)

In [ ]:
dur = ["30", "30", "30", "30", "30", "30", "30", "30", "30", "30", "30", "30"]
des = ["1", "1", "1", "2", "2", "2", "3", "3", "3", "4", "4", "4"]

my_annot = mne.Annotations(
    onset=raw_intensity.annotations.onset,
    duration=dur[0 : len(raw_intensity.annotations)],
    description=des[0 : len(raw_intensity.annotations)],
)
raw_intensity.set_annotations(my_annot)

In [ ]:
raw_intensity.plot(duration=len(raw_intensity))

### 3. Converting from raw intensity to optical density

In [ ]:
raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
raw_od.plot(duration=len(raw_intensity))

### 4. Data quality assessment

**The most important quality criterion for fNIRS data is a clearly visible heartbeat pattern in the signal.** Channels that lack this pattern and/or have a high number of artifacts should be removed either manually by visual inspection or automatically by calculating the **Scalp Coulpling Index (sci).** This method looks for the presence of a prominent synchronous signal (the heart rate) in the frequency range of the cardiac signals in both photodetected signals. The sci ranges from 0 (no optical transmission along an optical path) to 1 (complete optical transmission). All channels with sci value less than 0.5 should be removed. **We suggest a combined approach for the best possible result.**

#### Computing the scalp coupling index (sci)

In [ ]:
sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)
plt.figure()
plt.hist(sci)
plt.xlabel("Scalp Coupling Index")
plt.ylabel("Count")
plt.show()

#### Labeling all channes with an sci < 0.6 as bad

In [ ]:
raw_od.info["bads"] = raw_od.info["bads"] + list(compress(raw_od.ch_names, sci < 0.5))
print(f'Bad channels: {raw_od.info["bads"]}')

### 5. Artefact reduchtion
**Apply temporal derivative distribution repair**

This approach corrects baseline shift and spike artifacts without the need for any user-supplied parameters Fishburn et al.,2019.

*Frank A Fishburn, Ruth S Ludlum, Chandan J Vaidya, and Andrei V Medvedev. Temporal derivative distribution repair (tddr): a motion correction method for fNIRS. NeuroImage, 184:171–179, 2019. doi:10.1016/j.neuroimage.2018.09.025*


In [ ]:
corrected_od = mne.preprocessing.nirs.temporal_derivative_distribution_repair(raw_od)
corrected_od.plot(duration=len(raw_intensity))

### 5. Converting from optical density to haemoglobin
Convert the optical density data to haemoglobin concentration using the modified Beer-Lambert law.

In [ ]:
raw_haemo = mne.preprocessing.nirs.beer_lambert_law(corrected_od, ppf=0.1)
raw_haemo.plot(duration=len(raw_intensity))

### 6. Modelling
**Specify GLM: data, conditions, durations, hrf-model (e.g. glover or spm) and filter parameters as follows:**</br>
The cutoff period (1/high_pass) should be set as the longest period between two trials of the same condition multiplied by 2. For instance, if the longest period is 32s, the high_pass frequency shall be 1/64 Hz ~ 0.016 Hz. In the case of 30 seconds tapping and 30 seconds rest the inter-block-period is 60 secons*2 = 120 seconds --> 1/120 sec = 0.0083333 Hz 

In [ ]:
design_matrix = make_first_level_design_matrix(
    raw_haemo,
    drift_model="cosine",
    high_pass=0.008,  # Must be specified per experiment
    hrf_model="glover",
    stim_dur=30.0,
)

#### Displaying the design matrix

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6), nrows=1, ncols=1)
fig = plot_design_matrix(design_matrix, ax=ax1)
plt.show()

#### Fitting the model

In [ ]:
glm_est = run_glm(raw_haemo, design_matrix)

#### Displaying the results (only for good channels)

In [ ]:
cond = []
for ii in raw_haemo.annotations:
    if ii["description"] not in cond:
        cond.append(ii["description"])

glmReduced = glm_est.copy().pick(picks="hbo", exclude="bads")
fig, ax = plt.subplots(nrows=1, ncols=len(cond), figsize=(len(cond) * 5, 6))

if len(cond) > 1:
    for ii in range(0, len(cond)):
        glmReduced.copy().pick(picks=leftMotor).plot_topo(
            conditions=cond[ii], axes=ax[ii], colorbar=False, vlim=(-50, 50)
        )
        glmReduced.copy().pick(picks=rightMotor).plot_topo(
            conditions=cond[ii], axes=ax[ii], colorbar=False, vlim=(-50, 50)
        )
        glmReduced.copy().pick(picks=leftFrontal).plot_topo(
            conditions=cond[ii], axes=ax[ii], colorbar=False, vlim=(-50, 50)
        )
        glmReduced.copy().pick(picks=rightFrontal).plot_topo(
            conditions=cond[ii], axes=ax[ii], vlim=(-50, 50)
        )
else:
    glmReduced.copy().pick(picks=leftMotor).plot_topo(
        conditions=cond[0], axes=ax, colorbar=False, vlim=(-50, 50)
    )
    glmReduced.copy().pick(picks=rightMotor).plot_topo(
        conditions=cond[0], axes=ax, colorbar=False, vlim=(-50, 50)
    )
    glmReduced.copy().pick(picks=leftFrontal).plot_topo(
        conditions=cond[0], axes=ax, colorbar=False, vlim=(-50, 50)
    )
    glmReduced.copy().pick(picks=rightFrontal).plot_topo(
        conditions=cond[0], axes=ax, vlim=(-50, 50)
    )


plt.show()

#### Writing the condition- and channel-wise model parameter estimates to results table

In [ ]:
results = glmReduced.to_dataframe()
for ii in cond:
    resCond = results[results.Condition == ii]
    for jj in resCond.ch_name:
        if len(cond) > 1:
            fNIRSresHbO[jj + " " + condition + "(" + session + ii + ")"][
                fNIRSresHbO.SID == sid
            ] = resCond[resCond.ch_name == jj].theta.to_numpy()
        else:
            fNIRSresHbO[jj + " " + condition + "(" + session + ")"][
                fNIRSresHbO.SID == sid
            ] = resCond[resCond.ch_name == jj].theta.to_numpy()

fNIRSresHbO.to_excel("FTT_fNIRS_results.xlsx", index=False)